In [1]:
from sklearn.feature_selection import SelectKBest, f_classif, f_regression
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, r2_score
from datetime import datetime
import pandas as pd
import numpy as np
import pygal

In [2]:
def IV(df, var, tgt):
    aux = df[[var, tgt]].groupby(var).agg(["count", "sum"])
    aux["evento"] = aux[tgt, "sum"]
    aux["no_evento"] = aux[tgt, "count"] - aux[tgt, "sum"]
    aux["%evento"] = aux["evento"] / aux["evento"].sum()
    aux["%no_evento"] = aux["no_evento"] / aux["no_evento"].sum()
    aux["WOE"] = np.log(aux["%no_evento"] / aux["%evento"])
    aux["IV"] = (aux["%no_evento"] - aux["%evento"])*aux["WOE"]
    return aux["IV"].sum()

### Datos

In [3]:
df = pd.read_csv("/home/oscar/Downloads/OnlineNewsPopularity.csv").sample(frac = 0.1).reset_index(drop = True)

df.columns = [x.strip() for x in df.columns]

df["success"] = (df["shares"] > df["shares"].quantile(.5))*1

ls_cont = ['n_tokens_title', 'n_tokens_content', 'n_unique_tokens', 'n_non_stop_words', 
           'n_non_stop_unique_tokens', 'num_hrefs', 'num_self_hrefs', 'num_imgs', 'num_videos', 
           'average_token_length', 'num_keywords', 'kw_min_min', 'kw_max_min', 'kw_avg_min', 
           'kw_min_max', 'kw_max_max', 'kw_avg_max', 'kw_min_avg', 'kw_max_avg', 'kw_avg_avg', 
           'self_reference_min_shares', 'self_reference_max_shares', 'self_reference_avg_sharess', 
           'LDA_00', 'LDA_01', 'LDA_02', 'LDA_03', 'LDA_04', 'global_subjectivity', 
           'global_sentiment_polarity', 'global_rate_positive_words', 'global_rate_negative_words',
           'rate_positive_words', 'rate_negative_words', 'avg_positive_polarity', 'min_positive_polarity',
           'max_positive_polarity', 'avg_negative_polarity', 'min_negative_polarity', 
           'max_negative_polarity', 'title_subjectivity', 'title_sentiment_polarity', 
           'abs_title_subjectivity', 'abs_title_sentiment_polarity']
target = "shares"
target_disc = "success"

In [4]:
df

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares,success
0,http://mashable.com/2013/06/13/bird-apps-confu...,574.0,6.0,932.0,0.496696,1.0,0.677305,17.0,0.0,1.0,...,1.0,-0.222768,-0.666667,-0.050000,0.000000,0.000000,0.500000,0.000000,393,0
1,http://mashable.com/2013/09/16/vine-red-carpet/,479.0,15.0,195.0,0.645833,1.0,0.820513,6.0,3.0,0.0,...,0.5,-0.162500,-0.200000,-0.125000,0.000000,0.000000,0.500000,0.000000,924,0
2,http://mashable.com/2014/11/11/utah-ski-season/,56.0,13.0,296.0,0.624113,1.0,0.758242,3.0,2.0,0.0,...,0.5,0.000000,0.000000,0.000000,1.000000,0.500000,0.500000,0.500000,1000,0
3,http://mashable.com/2014/06/05/kittens-turn-do...,217.0,13.0,549.0,0.519409,1.0,0.701863,2.0,2.0,2.0,...,0.6,-0.362720,-1.000000,-0.050000,0.568182,-0.037879,0.068182,0.037879,1800,1
4,http://mashable.com/2013/01/23/wind-waker-hd-z...,715.0,11.0,611.0,0.433278,1.0,0.544959,1.0,0.0,1.0,...,1.0,-0.269246,-0.400000,-0.071429,0.454545,0.136364,0.045455,0.136364,6000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3959,http://mashable.com/2013/07/20/pets-like-us/,537.0,9.0,229.0,0.699115,1.0,0.877698,8.0,4.0,0.0,...,0.5,-0.241667,-0.666667,-0.125000,0.000000,0.000000,0.500000,0.000000,892,0
3960,http://mashable.com/2014/08/23/merkel-cease-fi...,137.0,7.0,166.0,0.713415,1.0,0.902174,5.0,3.0,1.0,...,0.5,-0.218519,-0.300000,-0.155556,0.000000,0.000000,0.500000,0.000000,1600,1
3961,http://mashable.com/2013/07/02/zynga-ceo-brief/,554.0,6.0,458.0,0.537585,1.0,0.726923,34.0,15.0,1.0,...,0.6,-0.307540,-0.500000,-0.050000,0.000000,0.000000,0.500000,0.000000,2300,1
3962,http://mashable.com/2014/06/16/dad-tattoo-reac...,206.0,9.0,303.0,0.594684,1.0,0.769634,12.0,5.0,1.0,...,0.6,-0.166667,-0.166667,-0.166667,0.000000,0.000000,0.500000,0.000000,1300,0


In [11]:
df["date"] = pd.to_datetime(df["url"].str.split("/").str[3:6].str.join("/"))
df["month"] = df["date"].dt.strftime("%Y-%m-01")

In [14]:
sorted(df["month"].unique())

['2013-01-01',
 '2013-02-01',
 '2013-03-01',
 '2013-04-01',
 '2013-05-01',
 '2013-06-01',
 '2013-07-01',
 '2013-08-01',
 '2013-09-01',
 '2013-10-01',
 '2013-11-01',
 '2013-12-01',
 '2014-01-01',
 '2014-02-01',
 '2014-03-01',
 '2014-04-01',
 '2014-05-01',
 '2014-06-01',
 '2014-07-01',
 '2014-08-01',
 '2014-09-01',
 '2014-10-01',
 '2014-11-01',
 '2014-12-01']

In [16]:
validate = df[df["date"] >= datetime(2014, 6, 1)].reset_index(drop = True).copy()
train = df.drop(index = validate.index).reset_index(drop = True).copy()

In [17]:
train = train[train[target] <= 32000].reset_index(drop = True)

In [18]:
Xt = train[ls_cont]
yr = train[target]
yc = train[target_disc]

### Modelado 

In [19]:
logreg = LogisticRegression()

In [20]:
logreg.fit(Xt, yc)

/home/oscar/VirtualEnvs/Python3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [21]:
%%time
ls_score = cross_val_score(estimator=logreg, X = Xt, y=yc, n_jobs=-1, scoring="roc_auc", cv = 4)

CPU times: user 50.1 ms, sys: 42.8 ms, total: 92.9 ms
Wall time: 1.84 s


In [22]:
np.mean(ls_score), np.std(ls_score)

(0.6357567791036255, 0.016842879235541796)

In [23]:
validate["yc_hat"] = logreg.predict(validate[ls_cont])

In [24]:
linreg = Ridge()

In [25]:
linreg.fit(Xt, yr)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [26]:
%%time
ls_score = cross_val_score(estimator=linreg, X = Xt, y=yr, n_jobs=-1, scoring="r2", cv = 4)

CPU times: user 39.9 ms, sys: 24.2 ms, total: 64.1 ms
Wall time: 1.66 s


In [27]:
np.mean(ls_score), np.std(ls_score)

(0.037388863831898544, 0.009569933972763804)

In [28]:
validate["yr_hat"] = linreg.predict(validate[ls_cont])

### PSI

In [32]:
dc_bins = {}
for feat in ls_cont:
    dc_bins.update({feat: pd.cut(Xt[feat], bins=3, retbins=True)[1]})

In [33]:
len(ls_cont)

44

In [54]:
for feat in dc_bins.keys():
    for mes in sorted(validate["month"].unique()):
        aux_t = pd.cut(Xt[feat], bins=dc_bins[feat]).value_counts(True).reset_index().rename(columns = {feat: f"{feat}_train"})
        aux_v = pd.cut(validate.loc[validate["month"] == mes, feat], bins=dc_bins[feat]).value_counts(True).reset_index().rename(columns = {feat: f"{feat}_valid"})
        aux = aux_t.merge(aux_v, on = "index")
        aux["diff"] = aux[f"{feat}_valid"] - aux[f"{feat}_train"]
        aux["log"] = np.log(aux[f"{feat}_valid"] / aux[f"{feat}_train"])
        aux["PSI"] = aux["diff"] * aux["log"]
        print(f'PSI of {feat} in {mes}: {aux["PSI"].sum()}')
    break

PSI of n_tokens_title in 2014-06-01: 0.05247938435646082
PSI of n_tokens_title in 2014-07-01: 0.00260371790649619
PSI of n_tokens_title in 2014-08-01: 0.05111806080083772
PSI of n_tokens_title in 2014-09-01: 0.03995400468245908
PSI of n_tokens_title in 2014-10-01: 0.08387750980904031
PSI of n_tokens_title in 2014-11-01: 0.12381160738896516
PSI of n_tokens_title in 2014-12-01: 0.16045489061955925


### Desempeño de poder predictivo 

In [56]:
for feat in ls_cont:
    for mes in sorted(validate["month"].unique()):
        kb = SelectKBest(k=1, score_func=f_classif)
        kb.fit(validate.loc[validate["month"] == mes, [feat]], validate.loc[validate["month"] == mes, "yc_hat"])
        print(f'Predictive power of {feat} in {mes}: {kb.scores_[0]}')
    break 

Predictive power of n_tokens_title in 2014-06-01: 0.7234079792749407
Predictive power of n_tokens_title in 2014-07-01: 0.45563402749461945
Predictive power of n_tokens_title in 2014-08-01: 1.1518257854592437
Predictive power of n_tokens_title in 2014-09-01: 4.358942832121547
Predictive power of n_tokens_title in 2014-10-01: 0.019103875101970938
Predictive power of n_tokens_title in 2014-11-01: 1.147513118973646
Predictive power of n_tokens_title in 2014-12-01: 0.8762861330233483


In [59]:
for feat in ls_cont:
    for mes in sorted(validate["month"].unique()):
        validate[f"C_{feat}"] = pd.cut(validate[feat], bins=dc_bins[feat])
        print(f'Predictive power of {feat} in {mes}: {IV(validate[validate["month"] == mes], f"C_{feat}", "yc_hat")}')
    break

Predictive power of n_tokens_title in 2014-06-01: 0.11714195335066217
Predictive power of n_tokens_title in 2014-07-01: 0.043928616456101105
Predictive power of n_tokens_title in 2014-08-01: 0.12588604539759102
Predictive power of n_tokens_title in 2014-09-01: 0.08063886186604585
Predictive power of n_tokens_title in 2014-10-01: 0.00874500893140444
Predictive power of n_tokens_title in 2014-11-01: 0.004765934999443226
Predictive power of n_tokens_title in 2014-12-01: 0.2219808976407348


### Estabilidad de las características 

In [60]:
for feat in dc_bins.keys():
    for mes in sorted(validate["month"].unique()):
        aux_t = pd.cut(Xt[feat], bins=dc_bins[feat]).value_counts(True).reset_index().rename(columns = {feat: f"{feat}_train"})
        aux_v = pd.cut(validate.loc[validate["month"] == mes, feat], bins=dc_bins[feat]).value_counts(True).reset_index().rename(columns = {feat: f"{feat}_valid"})
        aux = aux_t.merge(aux_v, on = "index")
        aux["diff"] = abs(aux[f"{feat}_valid"] - aux[f"{feat}_train"])
        print(f'Stability KS of {feat} in {mes}: {max(aux["diff"])}')
    break

Stability KS of n_tokens_title in 2014-06-01: 0.06049008400019881
Stability KS of n_tokens_title in 2014-07-01: 0.013162485100444807
Stability KS of n_tokens_title in 2014-08-01: 0.05744687036660173
Stability KS of n_tokens_title in 2014-09-01: 0.05034708433275617
Stability KS of n_tokens_title in 2014-10-01: 0.09669533784530565
Stability KS of n_tokens_title in 2014-11-01: 0.10830406795303982
Stability KS of n_tokens_title in 2014-12-01: 0.12047380585516179


### Performance 

In [61]:
for month in sorted(validate["month"].unique()):
    aux = validate[validate["month"] == month]
    print(month, roc_auc_score(y_score=aux["yc_hat"], y_true=aux["success"]))

2014-06-01 0.5893848288621648
2014-07-01 0.5769794721407625
2014-08-01 0.641566265060241
2014-09-01 0.5665382498624105
2014-10-01 0.6447080291970803
2014-11-01 0.5736386138613861
2014-12-01 0.627831715210356


In [63]:
for month in sorted(validate["month"].unique()):
    aux = validate[validate["month"] == month]
    print(month, r2_score(y_pred=aux["yr_hat"], y_true=aux["shares"]))

2014-06-01 0.17313847291681794
2014-07-01 0.002928026381499138
2014-08-01 0.0543089310363819
2014-09-01 0.030727747465078892
2014-10-01 0.05470464145138132
2014-11-01 -0.01204832346411111
2014-12-01 0.0359006434397906
